<a href="https://colab.research.google.com/github/Sayandeep27/Gen-AI-with-Sayandeep/blob/main/Human_Detection_with_Custom_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [ ]:
!kaggle datasets download constantinwerner/human-detection-dataset

Dataset URL: https://www.kaggle.com/datasets/constantinwerner/human-detection-dataset
License(s): CC0-1.0
 95% 248M/260M [00:01<00:00, 202MB/s]
100% 260M/260M [00:01<00:00, 169MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/human-detection-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:

image_dir_0 = '/content/human detection dataset/0'
image_dir_1 = '/content/human detection dataset/1'

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

def load_data(image_dir_0, image_dir_1, img_size=(224, 224)):
    images = []
    labels = []

    # Load images from the '0' directory (no human)
    for img_name in os.listdir(image_dir_0):
        img_path = os.path.join(image_dir_0, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size)
        images.append(img)
        labels.append(0)  # Label for no human

    # Load images from the '1' directory (human present)
    for img_name in os.listdir(image_dir_1):
        img_path = os.path.join(image_dir_1, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size)
        images.append(img)
        labels.append(1)  # Label for human

    # Convert images to numpy arrays and normalize
    images = np.array(images) / 255.0
    labels = np.array(labels)

    return images, labels

In [ ]:
images, labels = load_data(image_dir_0, image_dir_1)

In [ ]:
# Step 5: Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Step 6: Build your own CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Model summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Step 7: Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 89s 4s/step - accuracy: 0.6033 - loss: 0.6826 - val_accuracy: 0.5676 - val_loss: 0.6834
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 84s 4s/step - accuracy: 0.6317 - loss: 0.6470 - val_accuracy: 0.6324 - val_loss: 0.6797
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 136s 3s/step - accuracy: 0.6137 - loss: 0.6695 - val_accuracy: 0.6270 - val_loss: 0.7362
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.6975 - loss: 0.6320 - val_accuracy: 0.7081 - val_loss: 0.5895
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.7107 - loss: 0.5482 - val_accuracy: 0.7243 - val_loss: 0.5542
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 84s 4s/step - accuracy: 0.7976 - loss: 0.4623 - val_accuracy: 0.7405 - val_loss: 0.5609
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - accuracy: 0.8249 - loss: 0.3597 - val_accuracy: 0.7514 - val_loss: 0.5637
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.8849 - loss: 0.2719 - val_accuracy: 0.7405 - val_los

In [19]:
# Prediction function (same as before)
def predict_image(image_path):
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0
    img_input = np.expand_dims(img_normalized, axis=0)

    # Make prediction
    prediction = model.predict(img_input)
    return "Human detected!" if prediction[0][0] > 0.5 else "No human detected"


In [20]:
from google.colab import files
import cv2
import numpy as np

uploaded = files.upload()

image_path = next(iter(uploaded))

Saving 94.png to 94.png


In [21]:
# Step 3: Test the model with the uploaded image
result = predict_image(image_path)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
No human detected


In [22]:
from google.colab import files
import cv2
import numpy as np

uploaded = files.upload()

image_path = next(iter(uploaded))

Saving 89.png to 89.png


In [23]:
# Step 3: Test the model with the uploaded image
result = predict_image(image_path)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
No human detected


In [24]:
from google.colab import files
import cv2
import numpy as np

uploaded = files.upload()

image_path = next(iter(uploaded))

Saving 7.png to 7.png


In [25]:
# Step 3: Test the model with the uploaded image
result = predict_image(image_path)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
Human detected!
